# Waffle Hub Tutorial

# Prepare dataset

In [1]:
! tree mnist

mnist
├── coco.json
└── images
    ├── 100.png
    ├── 10.png
    ├── 11.png
    ├── 12.png
    ├── 13.png
    ├── 14.png
    ├── 15.png
    ├── 16.png
    ├── 17.png
    ├── 18.png
    ├── 19.png
    ├── 1.png
    ├── 20.png
    ├── 21.png
    ├── 22.png
    ├── 23.png
    ├── 24.png
    ├── 25.png
    ├── 26.png
    ├── 27.png
    ├── 28.png
    ├── 29.png
    ├── 2.png
    ├── 30.png
    ├── 31.png
    ├── 32.png
    ├── 33.png
    ├── 34.png
    ├── 35.png
    ├── 36.png
    ├── 37.png
    ├── 38.png
    ├── 39.png
    ├── 3.png
    ├── 40.png
    ├── 41.png
    ├── 42.png
    ├── 43.png
    ├── 44.png
    ├── 45.png
    ├── 46.png
    ├── 47.png
    ├── 48.png
    ├── 49.png
    ├── 4.png
    ├── 50.png
    ├── 51.png
    ├── 52.png
    ├── 53.png
    ├── 54.png
    ├── 55.png
    ├── 56.png
    ├── 57.png
    ├── 58.png
    ├── 59.png
    ├── 5.png
    ├── 60.png
    ├── 61.png
    ├── 62.png
    ├── 63.png
    ├── 64.png
    ├── 65.png
    ├── 66.png
    ├── 67.png
    ├── 68.pn

In [2]:
from waffle_utils.dataset import Dataset

ds = Dataset.from_coco(
    name="mnist_detection",
    coco_file="mnist/coco.json",
    coco_root_dir="mnist/images/"
)
! tree -d -L 1 datasets/mnist_detection

datasets/mnist_detection
├── categories
├── images
├── labels
└── raw

4 directories


In [3]:
ds.split_train_val(0.8)
! tree datasets/mnist_detection/sets

datasets/mnist_detection/sets
├── train.json
└── val.json

0 directories, 2 files


In [4]:
export_dir = ds.export("YOLO_DETECTION")
! tree -L 1 datasets/mnist_detection/exports/YOLO_DETECTION

datasets/mnist_detection/exports/YOLO_DETECTION
├── data.yaml
├── train
└── val

2 directories, 1 file


In [5]:
! cat datasets/mnist_detection/exports/YOLO_DETECTION/data.yaml

path: /home/lhj/ws/waffle_hub/datasets/mnist_detection/exports/YOLO_DETECTION
train: train
val: val
names:
    0: '1'
    1: '2'


# Create Hub

In [6]:
from waffle_hub import get_available_backends

print(get_available_backends())

┌─────────────┬───────────┐
│ Backend     │ Version   │
├─────────────┼───────────┤
│ ultralytics │ 8.0.49    │
│             │ 8.0.50    │
└─────────────┴───────────┘


/home/lhj/anaconda3/envs/play/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
from waffle_hub.hub import UltralyticsHub

hub = UltralyticsHub(
    name="my_model",
    task="detect",
    model_type="yolov8",
    model_size="n"
)

Model(name='my_model', backend='ultralytics', version='8.0.50', task='detect', model_type='yolov8', model_size='n')


In [8]:
! tree /home/lhj/ws/waffle_hub/models/my_model

/home/lhj/ws/waffle_hub/models/my_model
└── configs
    └── model.yaml

1 directory, 1 file


In [9]:
! cat /home/lhj/ws/waffle_hub/models/my_model/configs/model.yaml

name: my_model
backend: ultralytics
version: 8.0.50
task: detect
model_type: yolov8
model_size: n


In [10]:
model_dir = hub.train(
    dataset_dir=export_dir,
    epochs=30,
    batch_size=16,
    image_size=320
)

New https://pypi.org/project/ultralytics/8.0.51 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.0.50 🚀 Python-3.9.16 torch-1.13.1 CUDA:0 (NVIDIA GeForce RTX 4090, 24215MiB)
yolo/engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/home/lhj/ws/waffle_hub/datasets/mnist_detection/exports/YOLO_DETECTION/data.yaml, epochs=30, patience=50, batch=16, imgsz=320, save=True, save_period=-1, cache=False, device=0, workers=2, project=models/my_model, name=artifacts, exist_ok=False, pretrained=False, optimizer=SGD, verbose=True, seed=0, deterministic=True, single_cls=False, image_weights=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, min_memory=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, hide_labels=False, hide_conf=False, vid_stride=1, lin

In [11]:
! tree -L 1 /home/lhj/ws/waffle_hub/models/my_model

/home/lhj/ws/waffle_hub/models/my_model
├── artifacts
├── configs
├── metrics.csv
└── weights

3 directories, 1 file


In [12]:
! tree -L 1 /home/lhj/ws/waffle_hub/models/my_model/weights

/home/lhj/ws/waffle_hub/models/my_model/weights
├── best_ckpt.pt
└── last_ckpt.pt

0 directories, 2 files


In [13]:
hub = UltralyticsHub.load("my_model")

Model(name='my_model', backend='ultralytics', version='8.0.50', task='detect', model_type='yolov8', model_size='n')


In [14]:
inference_dir = hub.inference("mnist")


0: 320x320 1 1, 1: 320x320 1 2, 2: 320x320 1 1, 3: 320x320 1 2, 4: 320x320 1 2, 5: 320x320 1 1, 6: 320x320 1 1, 7: 320x320 1 2, 8: 320x320 1 1, 9: 320x320 1 2, 10: 320x320 1 2, 11: 320x320 1 2, 12: 320x320 1 1, 13: 320x320 1 1, 14: 320x320 1 2, 15: 320x320 1 1, 16: 320x320 1 2, 17: 320x320 1 2, 18: 320x320 1 1, 19: 320x320 1 2, 20: 320x320 1 2, 21: 320x320 1 2, 22: 320x320 1 2, 23: 320x320 1 2, 24: 320x320 1 2, 25: 320x320 1 2, 26: 320x320 1 1, 27: 320x320 1 2, 28: 320x320 1 1, 29: 320x320 1 2, 30: 320x320 1 1, 31: 320x320 1 1, 32: 320x320 1 1, 33: 320x320 1 1, 34: 320x320 1 2, 35: 320x320 1 1, 36: 320x320 1 2, 37: 320x320 1 2, 38: 320x320 1 2, 39: 320x320 1 2, 40: 320x320 1 1, 41: 320x320 1 2, 42: 320x320 1 1, 43: 320x320 1 1, 44: 320x320 1 2, 45: 320x320 1 2, 46: 320x320 1 1, 47: 320x320 1 1, 48: 320x320 1 1, 49: 320x320 1 2, 50: 320x320 1 2, 51: 320x320 1 2, 52: 320x320 1 1, 53: 320x320 1 2, 54: 320x320 1 1, 55: 320x320 1 1, 56: 320x320 1 1, 57: 320x320 1 1, 58: 320x320 1 1, 59: 32

In [16]:
! tree models/my_model/inferences | head

models/my_model/inferences
└── images
    ├── 100.json
    ├── 10.json
    ├── 11.json
    ├── 12.json
    ├── 13.json
    ├── 14.json
    ├── 15.json
    ├── 16.json


In [17]:
! cat models/my_model/inferences/images/100.json

{
    "image_path": "images/100.png",
    "predictions": [
        {
            "bbox": [
                98.5,
                117.0,
                19.0,
                46.0
            ],
            "class_name": "1",
            "confidence": 0.7963184714317322,
            "segment": []
        }
    ]
}